The aim of this notebook is to test `InferPy`'s potential on modelling Gaussian mixtures. In this case, `Mnist` dataset is used as the same outcome happends with both datasets.

Using this framework, this model cannot be learned due a limitation of `Tensorflow-Probability` where categorical variables cannot be learned using a gradient method. 


# Imports and dataset

In [1]:
# Imports
from functions import *
from models import mixture, Q_mixture
# Set seed
set_seed(2020)

Using TensorFlow backend.


Load and prepare Mnist database

In [2]:
# Load data from InferPy
(X, y), _ = mnist.load_data(num_instances=100, digits=[1,2,3])
# Create dataframe
dataset = pd.DataFrame(data=X)
dataset["number"] = y
# Split dataframe
X = dataset.drop(['number'], axis=1)
y = dataset['number']

# Constants

In [3]:
# n_components
n_components = len(y.unique())
# Ammount of samples
n_samples = X.shape[0]
# Observed space dimensionality
observed_space_dim = X.shape[1]

# InferPy

`TensorFlow-Probability` name categorical variables as NOT_REPARAMETERIZED , which, citing
its documentation means that “samples from the distribution are not fully reparameterized,
and straight-through gradients are either partially unsupported or are not supported at all”.
Show categorial variables are non-reparameterized.

In [4]:
x = inf.Categorical(probs=[0.5, 0.5])
print(x.reparameterization_type)

<Reparameteriation Type: NOT_REPARAMETERIZED>


We are trying to avoid them using a Mixture variable from `InferPy`.

Define the model object

In [5]:
model = mixture(n_components, observed_space_dim)

Define variational model

In [6]:
q = Q_mixture(n_components, observed_space_dim)

Define optimizer and inference object

In [7]:
# Define optimizer 
optimizer = tf.train.AdamOptimizer(0.01)
# Define inference object
VI = inf.inference.VI(q, optimizer=optimizer, epochs=2000)

The model is fitted using the dataset. This raises an error where the concentration parameter of the InverseGamma distribution reaches a non-positive value (even using a softplus function).

This being said, inference is not possible with this model and database.

In [8]:
model.fit({"x": X}, VI)


 0 epochs	 70896728.0....................
 200 epochs	 17031158.0....................
 400 epochs	 8986104.0....................
 600 epochs	 4760698.5....................
 800 epochs	 2697058.0....................
 1000 epochs	 2073698.5....................
 1200 epochs	 1506841.875....................
 1400 epochs	 1219866.875....................
 1600 epochs	 717736.125....

InvalidArgumentError: assertion failed: [Concentration must be positive.] [Condition x > 0 did not hold element-wise:] [x (Softplus_3:0) = ] [[3.79346776 3.7453568 4.28764]...]
	 [[node Lambda_8/assert_positive/assert_less/Assert/AssertGuard/Assert (defined at /home/ludvins/.pyenv/versions/3.6.9/envs/InferPy/lib/python3.6/site-packages/tensorflow_probability/python/distributions/inverse_gamma.py:169) ]]